In [1]:
import sys
sys.path.insert(0, '../../../network')

In [2]:
import logging
import argparse
import numpy as np
from network import Population, RateNetwork
from learning import ReachingTask
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence
import matplotlib.pyplot as plt
# np.set_printoptions(threshold=sys.maxsize)
logging.basicConfig(level=logging.INFO)

In [3]:
phi = ErrorFunction(mu=0.22, sigma=0.1).phi
ctx = Population(N=3000, tau=1e-2, phi=phi, name='ctx')
bg = Population(N=3000, tau=1e-2, phi=phi, name='bg')
plasticity = ThresholdPlasticityRule(x_f=0.5, q_f=0.8) 

S, P = 1, 4
sequences_ctx = [GaussianSequence(P,ctx.size, seed=11) for i in range(S)]
patterns_ctx = np.stack([s.inputs for s in sequences_ctx])
sequences_bg = [GaussianSequence(P,ctx.size, seed=367) for i in range(S)]
patterns_bg = np.stack([s.inputs for s in sequences_bg])

J_cc = SparseConnectivity(source=ctx, target=ctx, p=0.5)
synapse_cc = LinearSynapse(J_cc.K, A=6)
J_cc.store_attractors(patterns_ctx[0], patterns_ctx[0], synapse_cc.h_EE, plasticity.f, plasticity.g)
J_bb = SparseConnectivity(source=bg, target=bg, p=0.5)
synapse_bb = LinearSynapse(J_bb.K, A=6)
J_bb.store_attractors(patterns_bg[0], patterns_bg[0], synapse_bb.h_EE, plasticity.f, plasticity.g)
J_cb  = SparseConnectivity(source=bg, target=ctx, p=0.5)
synapse_cb = LinearSynapse(J_cb.K, A=5.5)
J_cb.store_attractors(patterns_bg[0], patterns_ctx[0], synapse_cb.h_EE, plasticity.f, plasticity.g)

J_bc = SparseConnectivity(source=ctx, target=bg, p=0.5)
synapse_bc = LinearSynapse(J_bc.K, A=0)
J_bc.store_sequences(patterns_ctx, patterns_bg, synapse_bc.h_EE, plasticity.f, plasticity.g)

net_ctx = RateNetwork(ctx, c_EE=J_cc, c_IE=J_bc, formulation=4, disable_pbar=True)
net_bg = RateNetwork(bg, c_II=J_bb, c_EI=J_cb, formulation=4)

INFO:connectivity:Building connections from ctx to ctx
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 3473.25it/s]
INFO:connectivity:Building connections from bg to bg
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 3518.24it/s]
INFO:connectivity:Building connections from bg to ctx
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 3430.86it/s]
INFO:connectivity:Building connections from ctx to bg
INFO:connectivity:Storing sequences
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.06it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix


In [ ]:
init_input_ctx = np.random.RandomState().normal(0,1,size=patterns_ctx[0][0].shape)
init_input_bg = np.random.RandomState().normal(0,1,size=patterns_bg[0][0].shape)
# init_input_ctx = patterns_ctx[0][2]
# init_input_bg = patterns_bg[0][2]
T=50
mouse = ReachingTask(2, alpha=0.2)
behaviors = net_ctx.simulate_learning(mouse, net_bg, T, init_input_ctx, init_input_bg, patterns_ctx[0], patterns_bg[0], plasticity, noise=20)

INFO:network:Integrating network dynamics


updating


/Users/stanleypark/.pyenv/versions/3.8.13/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


updating
updating
lick-->back out 0 0
back-->aim out 0 0
aim-->reach in 0 0
reach-->lick in 1 0.199998
lick-->aim in 0 0.019976400000000005
aim-->reach in 0 0.03596164000000001
reach-->lick in 1 0.36359640000000004
lick-->back in 0 0.035976400000000006
back-->aim out 0 0.006435160000000001
aim-->reach in 0 0.09419495200000003
reach-->lick in 1 0.497350872
lick-->back in 0 0.0499194488
back-->aim out 0 0.022081219360000008
aim-->reach in 0 0.16485711856000004
reach-->lick in 1 0.6068641983840001
lick-->back in 0 0.0638901785248
back-->aim out 0 0.047317256828800024
aim-->reach in 0 0.13479157836448005
reach-->back in 1 0.012990815628800006
back-->reach out 1 0
reach-->lick out 0 0
lick-->back out 0 0
back-->aim out 0 0.06170626071744642
aim-->reach in 0 0.11014960950476803
reach-->back in 1 0.010370292503040003
back-->aim out 1 0.01870772650547201
aim-->reach in 0 0.19140914486750724
reach-->lick in 1 0.692301809215296


In [ ]:
init_input_ctx = np.random.RandomState().normal(0,1,size=patterns_ctx[0][0].shape)
init_input_bg = np.random.RandomState().normal(0,1,size=patterns_bg[0][0].shape)
T=10
net_ctx.simulate_euler2(net_bg, T, init_input_ctx, init_input_bg, noise=10)

In [ ]:
overlaps = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi)
correlations = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi, correlation=True)

In [ ]:
for i, row in enumerate(correlations):
    plt.plot(row, label=str(i))
    plt.xlim(25000, 35000)
plt.legend()

In [ ]:
net_ctx.W.toarray().shape
